In [6]:
import cv2
import pytesseract
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set the path to the Tesseract OCR executable
pytesseract.pytesseract.tesseract_cmd = r'/path/to/tesseract'  # Update this to your Tesseract path

def extract_channel_name(image_path):
    image = cv2.imread(image_path)
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray_image)
    return text.strip()

def get_authenticated_service():
    client_secret_file = 'client_secret.json'  # Update this with your client_secret.json file path
    scopes = ['https://www.googleapis.com/auth/youtube.force-ssl']

    flow = InstalledAppFlow.from_client_secrets_file(client_secret_file, scopes)
    credentials = flow.run_console()

    return build('youtube', 'v3', credentials=credentials)

def search_and_subscribe_channel(channel_name, youtube):
    try:
        search_response = youtube.search().list(
            q=channel_name,
            type='channel',
            part='id,snippet',
            maxResults=1
        ).execute()

        if search_response['items']:
            channel_id = search_response['items'][0]['id']['channelId']
            subscribe_response = youtube.subscriptions().insert(
                part='snippet',
                body={
                    'snippet': {
                        'resourceId': {
                            'kind': 'youtube#channel',
                            'channelId': channel_id
                        }
                    }
                }
            ).execute()
            print(f'Successfully subscribed to {channel_name} (channel_id: {channel_id})')
        else:
            print(f'No channel found for {channel_name}')

    except HttpError as error:
        print(f'An error occurred: {error}')

def main():
    image_path = 'channel_screenshot.png'  # Update this with your screenshot path
    channel_name = extract_channel_name(image_path)
    youtube = get_authenticated_service()
    search_and_subscribe_channel(channel_name, youtube)

if __name__ == '__main__':
    main()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [14]:
import os
import subprocess
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import sys
import subprocess


import youtube_dl

def download_video_audio(youtube_url, output_file):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '192',
        }],
        'outtmpl': output_file,
    }
    
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

def split_audio_into_chunks(audio_file):
    audio = AudioSegment.from_wav(audio_file)
    chunks = split_on_silence(audio, min_silence_len=500, silence_thresh=-32)
    return chunks

def transcribe_audio_chunks(chunks):
    recognizer = sr.Recognizer()
    transcriptions = []

    for i, chunk in enumerate(chunks):
        chunk.export("temp_chunk.wav", format="wav")
        with sr.AudioFile("temp_chunk.wav") as source:
            audio = recognizer.record(source)
            try:
                text = recognizer.recognize_google(audio)
                transcriptions.append(text)
                print(f"Chunk {i+1} Transcription: {text}")
            except Exception as e:
                print(f"Error transcribing chunk {i+1}: {e}")

    return " ".join(transcriptions)

def main():
    youtube_url = "https://www.youtube.com/watch?v=thARc5J3l0k"  # Replace with the YouTube video URL
    output_file = "./output_audio.mp3"
    download_video_audio(youtube_url, output_file)
    audio_chunks = split_audio_into_chunks(output_file)
    transcription = transcribe_audio_chunks(audio_chunks)
    print("\nComplete Transcription:")
    print(transcription)

    if os.path.exists("temp_chunk.wav"):
        os.remove("temp_chunk.wav")

if __name__ == "__main__":
    main()


[youtube] thARc5J3l0k: Downloading webpage


ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.


DownloadError: ERROR: Unable to extract uploader id; please report this issue on https://yt-dl.org/bug . Make sure you are using the latest version; see  https://yt-dl.org/update  on how to update. Be sure to call youtube-dl with the --verbose flag and include its complete output.